In [1]:
import sys
sys.path.append('../')

In [2]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [168]:
from utils.backtest import calc_backtest
from utils.data_loaders import load_clc_db_records
from utils.features import construct_features_batch
from utils.data_handling import merge_asset_data, split_by_date
from utils.loss_functions import return_loss, sharpe_loss, return_loss_dummy, sharpe_loss_dummy

In [4]:
config = tf.ConfigProto(device_count = {'GPU': 1 , 'CPU': 8}) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [5]:
SIGMA_TARGET = 0.15

In [6]:
RAD_DATA_PATH = '../data/clc/rad/'
FED_DATA_PATH = '../data/FEDFUNDS.csv'

In [7]:
clc = load_clc_db_records(RAD_DATA_PATH, FED_DATA_PATH)

In [8]:
clc = construct_features_batch(clc)
for asset, df in clc.items():
    df.set_index('Date', inplace=True)

../utils/features.py:52: RuntimeWarning: invalid value encountered in log
  df['Sigma_Norm'] = np.log(df['Sigma'] / df['Sigma'].rolling(181).mean())


In [84]:
df = clc['SC']

In [85]:
df = df['1990-1-1':'2020-1-1']

In [169]:
calc_backtest(df.Returns_Daily, df.Excess_Returns_Daily, pd.Series(1, index=df.index))

E[Return]                         0.068347
E[Excess Return]                  0.041899
Std[Return]                       0.185134
Skew[Return]                      0.046243
Exc.Kurtosis[Return]             11.106244
Sharpe                            0.226317
Max_Drawdown                     -0.628857
Max_Drawdown_Duration (Years)    13.305556
dtype: float64

In [164]:
monthly_returns = ((1 + df.Excess_Returns_Daily).resample('M').prod() - 1).rename('Monthly_Returns')
yearly_returns = ((1 + df.Excess_Returns_Daily).rolling(252).apply(np.prod, raw=True) - 1).rename('Yearly_Returns')
vola = np.sqrt(252) * df['Sigma']
sign_data = pd.DataFrame([monthly_returns, yearly_returns, vola]).T
sign_data.dropna(inplace=True)
sign_data['SIGN'] = 2 * (sign_data['Yearly_Returns'] >= 0) - 1
sign_data['Next_Monthly_Returns'] = sign_data['Monthly_Returns'].shift(-1)
sign_data['Strategy_Weight'] = sign_data['SIGN'] * SIGMA_TARGET / data['Sigma']
sign_data['Performance'] = sign_data['Strategy_Weight'] * sign_data['Next_Monthly_Returns']

In [170]:
calc_backtest(df.Returns_Daily, df.Excess_Returns_Daily, sign_data.Strategy_Weight)

E[Return]                        0.067765
E[Excess Return]                 0.050834
Std[Return]                      0.163805
Skew[Return]                    -0.594898
Exc.Kurtosis[Return]             5.780940
Sharpe                           0.310335
Max_Drawdown                    -0.450400
Max_Drawdown_Duration (Years)    3.507937
dtype: float64